In [5]:
import asyncio
from playwright.async_api import async_playwright
from urllib.parse import urljoin
from pyquery import PyQuery as pq
import time
import json
import re
from motor.motor_asyncio import AsyncIOMotorClient
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

In [7]:
BASE_URL = 'https://www.qzsc.cc/'
cat_name = {
    'xuanhuanxiaoshuo/': '玄幻',
    'xiuzhenxiaoshuo/': '修真',
    'dushixiaoshuo/': '都市',
    }

In [8]:
for key, value in cat_name.items():
    print(key, value)
    print(type(key), type(value))
    url = urljoin(BASE_URL, key)
    print(url)

xuanhuanxiaoshuo/ 玄幻
<class 'str'> <class 'str'>
https://www.qzsc.cc/xuanhuanxiaoshuo/
xiuzhenxiaoshuo/ 修真
<class 'str'> <class 'str'>
https://www.qzsc.cc/xiuzhenxiaoshuo/
dushixiaoshuo/ 都市
<class 'str'> <class 'str'>
https://www.qzsc.cc/dushixiaoshuo/
